<a href="https://colab.research.google.com/github/pysac17/tinder-clone/blob/master/TimeGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, GRU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError, BinaryCrossentropy
from sklearn.preprocessing import MinMaxScaler
import tensorflow.keras.backend as K
from tensorflow.data import Dataset as tfdata
from numpy import sqrt, abs
from numpy.random import permutation, uniform
from numpy import ones_like, zeros_like
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
df = pd.read_csv('sml2010.csv')
data = df.values

FileNotFoundError: [Errno 2] No such file or directory: 'sml2010.csv'

In [ ]:
seq_len = 24
n_seq = 6
hidden_dim = 24
gamma = 1
noise_dim = 32
dim = 128
batch_size = 128
learning_rate = 5e-4
train_steps = 5000

In [ ]:
def preprocess(data, seq_len):
    ori_data = data[::-1]
    scaler = MinMaxScaler().fit(ori_data)
    ori_data = scaler.transform(ori_data)

    temp_data = []
    for i in range(0, len(ori_data) - seq_len):
        _x = ori_data[i:i + seq_len]
        temp_data.append(_x)

    idx = permutation(len(temp_data))
    data = []
    for i in range(len(temp_data)):
        data.append(temp_data[idx[i]])

    return data

In [ ]:
def net(model, n_layers, hidden_units, output_units, net_type='GRU'):
    for i in range(n_layers):
        if net_type == 'GRU':
            model.add(GRU(units=hidden_units, return_sequences=True, name=f'GRU_{i + 1}'))
        elif net_type == 'LSTM':
            model.add(LSTM(units=hidden_units, return_sequences=True, name=f'LSTM_{i + 1}'))

    model.add(Dense(units=output_units, activation='sigmoid', name='OUT'))
    return model

In [ ]:
class Generator(Model):
    def __init__(self, hidden_dim, net_type='GRU'):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.net_type = net_type

    def build(self, input_shape):
        model = Sequential(name='Generator')
        model = net(model, n_layers=3, hidden_units=self.hidden_dim, output_units=self.hidden_dim, net_type=self.net_type)
        return model

In [ ]:
class Supervisor(Model):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim

    def build(self, input_shape):
        model = Sequential(name='Supervisor')
        model = net(model, n_layers=2, hidden_units=self.hidden_dim, output_units=self.hidden_dim)
        return model


In [ ]:
class Discriminator(Model):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim

    def build(self, input_shape):
        model = Sequential(name='Discriminator')
        model = net(model, n_layers=3, hidden_units=self.hidden_dim, output_units=1)
        return model

In [ ]:
class Recovery(Model):
    def __init__(self, hidden_dim, n_seq):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_seq = n_seq

    def build(self, input_shape):
        model = Sequential(name='Recovery')
        model = net(model, n_layers=3, hidden_units=self.hidden_dim, output_units=self.n_seq)
        return model

In [ ]:
class Embedder(Model):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim

    def build(self, input_shape):
        model = Sequential(name='Embedder')
        model = net(model, n_layers=3, hidden_units=self.hidden_dim, output_units=self.hidden_dim)
        return model

In [ ]:
class TimeGAN(Model):
    def __init__(self, model_parameters, hidden_dim, seq_len, n_seq, gamma):
        super().__init__()
        self.seq_len = seq_len
        self.batch_size, self.lr, self.beta_1, self.beta_2, self.noise_dim, self.data_dim, self.layers_dim = model_parameters
        self.n_seq = n_seq
        self.hidden_dim = hidden_dim
        self.gamma = gamma
        self.define_gan()

    def define_gan(self):
        self.generator_aux = Generator(self.hidden_dim).build(input_shape=(self.seq_len, self.n_seq))
        self.supervisor = Supervisor(self.hidden_dim).build(input_shape=(self.hidden_dim, self.hidden_dim))
        self.discriminator = Discriminator(self.hidden_dim).build(input_shape=(self.hidden_dim, self.hidden_dim))
        self.recovery = Recovery(self.hidden_dim, self.n_seq).build(input_shape=(self.hidden_dim, self.hidden_dim))
        self.embedder = Embedder(self.hidden_dim).build(input_shape=(self.seq_len, self.n_seq))


In [ ]:
gan_args = (batch_size, learning_rate, noise_dim, 24, 2, (0, 1), dim)
synth = TimeGAN(model_parameters=gan_args, hidden_dim=24, seq_len=seq_len, n_seq=n_seq, gamma=1)